In [1]:
from mstr_robotics.report import rep, prompts, cube
from mstr_robotics.user_RAG import keyword_processor, vectorDB_faisst,mstr_openAI,chat_bot, perplexity
import pandas as pd
from mstrio.connection import Connection
from mstr_robotics.user_RAG import keyword_processor
from mstr_robotics.navigation import mstr_objects
import json 

user_path="..\\config\\user_d.json"
with open(user_path, 'r') as file:
    user_d = json.load(file)

jupyter_path="..\\config\\jupyter_objects_d.json"
with open(jupyter_path, 'r') as file:
    jupyter_objects_d = json.load(file)

conn_params =  user_d["conn_params"]
sKey=user_d["sKey"]

#GUID of report wizzard & prompts
project_id=jupyter_objects_d["turtorial_RAG"]["project_id"]          #Tutorial project
#Mstr prompted report
rep_dos_id=jupyter_objects_d["chat_bot_rep"]["rep_dos_id"]
#GUID of output folder for generated reports
ai_rep_folder_id = jupyter_objects_d["chat_bot_rep"]["ai_rep_folder_id"]

#cubes generated in load_rag_cubes
RAG_Process="turtorial_RAG"
cube_attribute_form_elements_name=jupyter_objects_d[RAG_Process]["cube_attribute_form_elements_name"]
cube_attribute_form_elements_id=jupyter_objects_d[RAG_Process]["cube_attribute_form_elements_id"]

cube_attribute_elements_id=jupyter_objects_d[RAG_Process]["cube_attribute_elements_id"]
cube_attribute_elements_name=jupyter_objects_d[RAG_Process]["cube_attribute_elements_name"]

cube_obj_prp_rel_name=jupyter_objects_d[RAG_Process]["cube_obj_prp_rel_name"]
cube_obj_prp_rel_id=jupyter_objects_d[RAG_Process]["cube_obj_prp_rel_id"] 


cube_att_form_def_name=jupyter_objects_d[RAG_Process]["cube_att_form_def_name"]
cube_att_form_def_id= jupyter_objects_d[RAG_Process]["cube_att_form_def_id"] 
#cube_att_form_def_id=None

cbe_dashboard_definitions_id=jupyter_objects_d[RAG_Process]["cbe_dashboard_definitions_id"] 
cbe_dashboard_definitions_name=jupyter_objects_d[RAG_Process]["cbe_dashboard_definitions_name"] 

cbe_dashboard_chapter_filt_id=jupyter_objects_d[RAG_Process]["cbe_dashboard_chapter_filt_id"] 
cbe_dashboard_chapter_filt_name=jupyter_objects_d[RAG_Process]["cbe_dashboard_chapter_filt_name"] 

cbe_dashboard_selector_filt_id=jupyter_objects_d[RAG_Process]["cbe_dashboard_selector_filt_id"] 
cbe_dashboard_selector_filt_name=jupyter_objects_d[RAG_Process]["cbe_dashboard_selector_filt_name"] 



cbe_cube_read_out_id=jupyter_objects_d[RAG_Process]["cbe_cube_read_out_id"] 
cbe_cube_read_out_name=jupyter_objects_d[RAG_Process]["cbe_cube_read_out_name"] 


cbe_rep_def_id= jupyter_objects_d[RAG_Process]["cbe_rep_def_id"] 
cbe_rep_def_name=jupyter_objects_d[RAG_Process]["cbe_rep_def_name"] 

cbe_tbl_att_fct_mapping_id= jupyter_objects_d[RAG_Process]["cbe_tbl_att_fct_mapping_id"] 
cbe_tbl_att_fct_mapping_name=jupyter_objects_d[RAG_Process]["cbe_tbl_att_fct_mapping_name"]  


os_mcp_folder_str="D:\\shared_drive\\mstr_output\\MCP_data"

i_keyword_processor=keyword_processor()
i_perplexity=perplexity()

In [2]:
conn_params["project_id"]=project_id
conn = Connection(**conn_params)
conn.headers['Content-type'] = "application/json"

Connection to MicroStrategy Intelligence Server has been established.


In [3]:
import pandas as pd
import os
csv_files_d_l=[ {"csv_file_name":"attribute_form_elements","key_col_l":[{"col_name":"element_val","new_col_name":"key"
                                                               , "key_type":"element_val"}]}
                ,{"csv_file_name":"obj_prp_rel","key_col_l":[{"col_name":"object_name","new_col_name":"key"
                                                             , "key_type":"bi_object"}]}
                ,{"csv_file_name":"attribute_elements","key_col_l":[{"col_name":"element_val","new_col_name":"key"
                                                                     , "key_type":"element_val"}]}
                ,{"csv_file_name":"dashboard_definitions","key_col_l":[{"col_name":"object_name", "new_col_name":"key",
                                                                           "key_type":"bi_object"},
                                                                           {"col_name":"chapter_name", "new_col_name":"chapter_name_key",
                                                                           "key_type":"chapter_name"},
                                                                           {"col_name":"page_name", "new_col_name":"page_name_key",
                                                                           "key_type":"page_name"},
                                                                           {"col_name":"visual_name", "new_col_name":"visual_name_key",
                                                                           "key_type":"visual_name"},]}
                
                ,{"csv_file_name":"dashboard_chapter_filter","key_col_l":[{"col_name":"target_object_name",
                                                                        "new_col_name":"key",
                                                                           "key_type":"object_name"}]}
                ,{"csv_file_name":"dashboard_selector_filter","key_col_l":[{"col_name":"target_object_name",
                                                                     "new_col_name":"key",
                                                                  "key_type":"object_name"}]}    
                
                                                    
                ]
key_l=[]
for csv in csv_files_d_l:
        print(csv["csv_file_name"])
        csv_file_name = csv["csv_file_name"]+ ".csv" 
        file_path = os.path.join(os_mcp_folder_str, csv_file_name)

        df=pd.read_csv(file_path)
        for key_col in csv["key_col_l"]:
                key_l.extend(df[key_col["col_name"]].astype(str).unique().tolist())
        

        
        key_l=list(set(key_l))  # Remove duplicates
i_keyword_processor.load_keyword_processor(key_l)

attribute_form_elements_df = pd.read_csv(os.path.join(os_mcp_folder_str, "attribute_form_elements.csv"))
attribute_elements_df = pd.read_csv(os.path.join(os_mcp_folder_str, "attribute_elements.csv"))

obj_prp_rel_df = pd.read_csv(os.path.join(os_mcp_folder_str, "obj_prp_rel.csv"))
dashboard_definitions_df = pd.read_csv(os.path.join(os_mcp_folder_str, "dashboard_definitions.csv"))
dashboard_chapter_filter_df = pd.read_csv(os.path.join(os_mcp_folder_str, "dashboard_chapter_filter.csv"))
dashboard_selector_filter_df = pd.read_csv(os.path.join(os_mcp_folder_str, "dashboard_selector_filter.csv"))

attribute_form_elements
obj_prp_rel
attribute_elements
dashboard_definitions
dashboard_chapter_filter
dashboard_selector_filter


In [4]:
def rem_dbl_dict_in_l (dict_l):

    tuple_l = set(tuple(sorted(d.items())) for d in dict_l)
    unique_dict_l = [dict(t) for t in tuple_l]

    return unique_dict_l

def get_att_elem_str(element_df_d_l, key_word_l):

    element_rag_d_l = []
    for df in element_df_d_l:
        element_df = df["df"]
        filt_df=element_df[df["rag_cols"]][element_df[df["key_col"]].isin(key_word_l)]
        element_rag_d_l.extend(filt_df.to_dict(orient='records'))

    element_rag_d_l= rem_dbl_dict_in_l(element_rag_d_l)
    return element_rag_d_l

def get_bi_obj_str(obj_df_d_l, key_word_l):
    obj_rag_d_l = []
    for df in obj_df_d_l:
        obj_df = df["df"]
        filt_df=obj_df[obj_df[df["key_col"]].isin(key_word_l) |obj_df[df["key_col"]].isin(key_word_l) ]
        obj_rag_d_l.extend(filt_df.to_dict(orient='records'))

    obj_rag_d_l= rem_dbl_dict_in_l(obj_rag_d_l)
    return obj_rag_d_l

In [ ]:
element_df_d_l=[{"df":attribute_form_elements_df,"key_col":"element_val","key_type":"element_val","rag_cols": ["attribute_name", "form_name", "element_val"]},
                 {"df":attribute_elements_df,"key_col":"element_val","key_type":"element_val","rag_cols": ["attribute_name", "element_val"]}
                ]

bi_obj_df=obj_prp_rel_df[["object_name", "obj_type", "object_id"]][obj_prp_rel_df["obj_type"].isin(["attribute","metric"]) ]

obj_df_d_l=[{"df":bi_obj_df,"key_col":"object_name","key_type":"object_name","rag_cols": ["object_name", "obj_type"]}]
            



In [6]:
#rep_dos_id="01274D824D6798C5F5BBB3BD4D15BA2C"
msg_t="Please show me the Revenue, Cost and Profit for the attributes Category, Call Center"
msg_t= msg_t + " and filter for the yaer 2021, 2022"
msg_t= msg_t + " and Call Center Atlanta and Categories starting with B and Revenue is between 200 and 100000"
#msg_t="I need Revenue for the Category Books and Music"

key_word_l=i_keyword_processor.extract_keywords(msg_t=msg_t)

att_elem_str=get_att_elem_str(element_df_d_l, key_word_l=key_word_l)
bi_obj_str=get_att_elem_str(obj_df_d_l, key_word_l=key_word_l)
sys_cont=i_perplexity.rag_sys_cont(key_word_l=key_word_l,att_elem_str=att_elem_str, bi_obj_str=bi_obj_str)

message_check_d={}
message_check_d_l=[]
message_check_d["msg_nr"] = "1"
message_check_d["msg_t"] = msg_t
message_check_d=i_perplexity.call_perplexity( msg_t=msg_t, sys_cont=sys_cont, message_check_d=message_check_d, temperature=0.1)
message_check_d_l.append(message_check_d.copy())

#message_check_d["step"] = "template"
#message_check_d = i_perplexity.call_perplexity(msg_t=msg_t,message_check_d=message_check_d, sys_cont=sys_cont)
#message_check_d_l.append(message_check_d.copy())
bi_request_d=i_perplexity.parse_and_structure(message_check_d_l)
bi_request_d

dict_keys(['id', 'choices', 'created', 'model', 'object', 'service_tier', 'system_fingerprint', 'usage', 'citations', 'search_results'])


{'msg_nr': '1',
 'msg_t': 'Please show me the Revenue, Cost and Profit for the attributes Category, Call Center and filter for the yaer 2021, 2022 and Call Center Atlanta and Categories starting with B and Revenue is between 200 and 100000',
 'attributes': ['Category', 'Call Center'],
 'metrics': ['Revenue', 'Cost', 'Profit'],
 'filter': {'att_element_1': {'attribute': 'Year',
   'operator': 'In',
   'element_list': ['2021', '2022']},
  'att_element_2': {'attribute': 'Call Center',
   'operator': 'In',
   'element_list': ['Atlanta']},
  'att_qual_1': {'attribute': 'Category',
   'Column': 'desc',
   'operator': 'BeginsWith',
   'value': 'B'},
  'metric_filter_1': {'level': None,
   'metric': 'Revenue',
   'operator': 'Between',
   'value': [200, 100000]}},
 'question': ''}

In [ ]:
print("HHH")
template_obj_d_l=[]
filter_d_l=[]
for att in bi_request_d["attributes"]:
    template_obj_l.append({"object_name":att,"obj_type":"attribute"})
for metric in bi_request_d["metrics"]:
    template_obj_l.append({"object_name":metric,"obj_type":"metric"})

for filt in bi_request_d["filter"].keys():
    if f[:9] == "att_eleme":
        filter_d_l.append({"object_name":filt["attribute"],"obj_type":"attribute"})

    if f[:9] == "att_qual_":
        filter_d_l.append({"object_name":filt["attribute"],"obj_type":"attribute"})

    if f[:9] == "metric_fi":
        filter_d_l.append({"object_name":filt["metric"],"obj_type":"metric"})


print(template_obj_l)
print(filter_d_l)
print("HHH")

In [ ]:
len(key_l)

In [ ]:
print("HHH")
filter_att_l=[]
filter_met_l=[]

for filt in bi_request_d["filter"].keys():
    if filt[:9] == "att_eleme":
        filter_att_l.append(bi_request_d["filter"][filt]["attribute"])

    if filt[:9] == "att_qual_":
        filter_att_l.append(bi_request_d["filter"][filt]["attribute"])

    if filt[:9] == "metric_fi":
        filter_met_l.append(bi_request_d["filter"][filt]["metric"])


print(filter_att_l)
print(filter_met_l)
print("HHH")
dashboard_chapter_filter_df.columns
target_object_name

In [ ]:
obj_prp_rel_df
template_obj_df= dashboard_definitions_df[
    ((dashboard_definitions_df["type"] == "attribute") |
    (dashboard_definitions_df["object_name"].isin(bi_request_d["attributes"]))) |
    ((dashboard_definitions_df["type"] == "attribute") |
    (dashboard_definitions_df["object_name"].isin(bi_request_d["attributes"]))) 
]
dash_cha_filt= dashboard_chapter_filter_df[
    ((dashboard_chapter_filter_df["target_object_type"] == "attribute") |
    (dashboard_chapter_filter_df["target_object_name"].isin(filter_att_l)))|
    ((dashboard_chapter_filter_df["target_object_type"] == "attribute") |
    (dashboard_chapter_filter_df["target_object_name"].isin(filter_att_l)))
]

dash_sel_filt= dashboard_selector_filter_df[
    ((dashboard_selector_filter_df["target_object_type"] == "attribute") |
    (dashboard_selector_filter_df["target_object_name"].isin(filter_att_l)))|
    ((dashboard_selector_filter_df["target_object_type"] == "attribute") |
    (dashboard_selector_filter_df["target_object_name"].isin(filter_att_l)))
]
dash_sel_filt

In [ ]:
dashboard_chapter_filter_df

In [ ]:
obj_prp_rel_df[""]